In [1]:
import pandas as pd
import numpy as np

In [5]:
import os
import re
os.chdir("../")

In [6]:
pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", 100)

## Analyze data

In [7]:
import pandas as pd
data = pd.read_parquet("data/prepared_data/prepared_df.parquet")

In [8]:
data.sample(10)

,place_id,review,question,tags,proba
32435,51,"Памятник сразу бросается в глаза, трость наводит на мысли, неужели, Чехову, но когда подойдешь б...",Для кого?,[подвиги безымянных героев],"[0.504, 0.282]"
36594,100,"Всё замечательно, вкусно, весело!!! Всё понравилось!, особенно клоун! И официант Мария хорошо и ...",Почему?,None,None
5844,1,"После того, что там было раньше, новая набережная это просто сказка! Грандиозные скульптуры, мно...",Почему?,"[красиво, вобщем, приятно, прогуляться]","[0.353, 0.274]"
3987,1,Лучшее место в центре города для прогулок и занятий спортом с апреля по октябрь. Особую атмосфер...,Что?,[лучшее место центре города прогулок занятий спортом],"[0.902, 0.33]"
32282,50,"Красивая мечеть в частном секторе. Внешне величественное сооружение, внутри всё уютно и чисто. В...",Когда?,None,None
35001,92,Первый (раньше был самый большой!) торговый центр в Тюмени! И сейчас пользуется популярностью. М...,Какой?,[первый ( раньше самый большой ! ) торговый центр тюмени],"[0.31, 0.268]"
5141,1,"Несмотря на ветреную весеннюю погоду, набережная понравилась. Чисто, красиво, много статуй, с ко...",Какой?,"[чисто, красиво]","[0.785, 0.607]"
7114,2,"По-правде говоря, мост появился задолго до набережной. Но, отстроив великолепную набережную, стр...",Почему?,[строителям пришлось привести порядок мост],"[0.366, 0.277]"
17934,7,Останавливались в Тюмени сразу после нового года и попали на премьеру. Актерский состав очень по...,Почему?,[очень профессионально],"[0.487, 0.176]"
27868,29,Ни каждый город России может похвастаться памятником маме. А в Тюмени он есть. Правда место по м...,Кому?,[памятником маме],"[0.486, 0.926]"


---

In [9]:
from src.models import roberta
from nltk.corpus import stopwords

/home/dmitry/projects/tags-by-review-nlp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
model = roberta.Pipeline()

In [97]:
def return_tags(part: dict, root_dep: str) -> str:
        tags = []
        # для того, чтобы оставить только главное существительное
        root_stop = False
        # для того, чтобы возврщать слова из предложения, а не леммы
        return_keys = False
        single_word = True if len(part.keys()) == 1 else False
        for num, key in enumerate(part.keys()):

            if single_word:
                if part[key]["pos"] == 'ADP':
                    continue
                tags.append(part[key]["lemma"])
                continue

            if root_stop:
                break

            # should return keys?
            if (
                num == 0 and (
                    (part[key]["dep"] == "case" or part[key]["dep"] == "advmod")
                    or (part[key]["dep"] == "nsubj")
                    or (part[key]["pos"] == 'CCONJ' and part[key]["dep"] == 'cc')
                    or (part[key]["dep"] == 'ROOT' and part[key]["pos"] != 'NOUN')
                    or (part[key]["pos"] == 'ADJ', part[key]["dep"] == 'amod')
                )
            ):
                return_keys = True

            # should pass the word?
            if (
                (
                    part[key]["pos"] == "ADV" 
                    and (
                        part[key]["dep"] == "advmod" 
                        or part[key]["dep"] == "punct"
                        or part[key]["dep"] == "ROOT"
                    )
                    and (num+1 != len(part.keys()) or num == 0)
                )
                or (part[key]["dep"] == ['nmod'] and num == 0)
                or (part[key]["pos"] == 'CCONJ' and part[key]["dep"] == 'cc' and num == 0)
                or (part[key]["dep"] == "mark" and root_dep == 'VERB')
                or (
                    part[key]["dep"] == 'iobj' 
                    and part[key]["pos"] == 'NOUN'
                    and num+1 != len(part.keys())
                )
                or (part[key]["pos"] == 'NOUN' and part[key]["dep"] == 'obj')
                or (part[key]["pos"] == 'DET' and part[key]["dep"] == 'det' and num == 0)
            ):
                continue

            if root_dep == 'VERB':
                if return_keys:
                    tags.append(key)
                else:
                    tags.append(part[key]["lemma"])

                if part[key]["dep"] == 'xcomp' and len(tags) == 1:
                    tags.pop(0)
                    tags.append(key)
            
            elif root_dep == "NOUN":
                if return_keys:
                    tags.append(key)
                    if (
                        num != 0 
                        and part[key]["dep"] == "ROOT"
                        and num != len(part.keys()) - 2
                    ):
                        root_stop = True
                else:
                    tags.append(part[key]["lemma"])

            elif  root_dep == "ADJ":
                if part[key]["dep"] == "mark":
                    continue
                else:
                    tags.append(key) 

            elif  root_dep == "ADV":
                tags.append(key)

            elif  root_dep == "NUM":
                tags.append(key)
            elif  root_dep == 'PRON':
                tags.append(key)
                              
            else:
                tags.append(part[key]["lemma"])

        return " ".join(tags)

In [98]:
questions = [
    "на что можно посмотреть в этом месте",
    "что понравилось в этом месте",
    "что делать в этом месте",
    "какие ожидания от этого места",
    "для кого это место"]

In [103]:
for i in data.sample(1).index:
    example = data.review[i]
    print(example, "\n")
    for question in questions:
        answer, probas = model.answer(example, question, 0.)

        if (
            not answer or
            answer == '' or
            answer == question.lower().replace('?', '')
        ):
            continue

        try:
            answer = answer.split(">")[1]
        except IndexError:
            pass
        # print(answer)

        stops = stopwords.words("russian")
        stops = [word for word in stops if word not in model._norm_words+["не", "на", "от", "а", "в", "за", "до", "с"]]

        corpus = answer.split(" ")
        text = " ".join([word for word in corpus if word not in stops])
        text_list = re.split(r', |\. |\.|,| а | и | просто | или |!|!!!| ! ', text)

        answer_cleared = text_list
        # print(text_list)

        parts = []
        roots_dep = []
        for answer in answer_cleared:
            document = model._nlp_model(answer)
            doc_dict = {}
            for token in document:
                if token.pos_ not in "PUNCT":
                    doc_dict[token.text] = {
                        "lemma": token.lemma_,
                        "pos": token.pos_,
                        "dep": token.dep_,
                    }
                    if token.dep_ == "ROOT":
                        roots_dep.append(token.pos_)
            parts.append(doc_dict)
        
        print(parts)

        tags = []
        for part, root_dep in zip(parts, roots_dep):
            if part:
                tags.append(return_tags(part, root_dep))
        
        print(question, tags, "\n", sep = "\n")

Самая красивая набережная! Четыре яруса. С подсветками. С петуньями. И петуньи разного цвета и аромата. И мост Влюбленных светится разными огнями. Велосипедисты. Артисты. Просто прохожие. Всем есть место. А также есть много интересных скульптур, рассказывающих историю Тюмени. Самое красивое место в городе. 

[{'интересных': {'lemma': 'интересный', 'pos': 'ADJ', 'dep': 'amod'}, 'скульптур': {'lemma': 'скульптура', 'pos': 'NOUN', 'dep': 'ROOT'}}]
на что можно посмотреть в этом месте
['интересных скульптур']


[{'интересных': {'lemma': 'интересный', 'pos': 'ADJ', 'dep': 'amod'}, 'скульптур': {'lemma': 'скульптура', 'pos': 'NOUN', 'dep': 'ROOT'}}]
что понравилось в этом месте
['интересных скульптур']


[{'велосипедисты': {'lemma': 'велосипедист', 'pos': 'NOUN', 'dep': 'ROOT'}}, {'артисты': {'lemma': 'артист', 'pos': 'NOUN', 'dep': 'ROOT'}}, {'просто': {'lemma': 'просто', 'pos': 'PART', 'dep': 'advmod'}, 'прохожие': {'lemma': 'прохожий', 'pos': 'NOUN', 'dep': 'ROOT'}}, {'всем': {'lemma': 'в